В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [25]:
from process_bank_churn import preprocess_data, preprocess_new_data
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
raw_df = pd.read_csv('train.csv')
data = preprocess_data(raw_df)
X_train = data['X_train']
train_targets = data['train_targets']
X_val = data['X_val']
val_targets = data['val_targets']

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [3]:
knn = KNeighborsClassifier()
knn.fit(X_train, train_targets)

targets_train_pred = knn.predict_proba(X_train)[:, 1]
targets_val_pred = knn.predict_proba(X_val)[:, 1]

train_auc = roc_auc_score(train_targets, targets_train_pred)
val_auc = roc_auc_score(val_targets, targets_val_pred)

print(f"AUROC на тренувальних даних: {train_auc:.4f}")
print(f"AUROC на валідаційних даних: {val_auc:.4f}")

AUROC на тренувальних даних: 0.8228
AUROC на валідаційних даних: 0.5783


**Висновки**

точність моделі AUROC на тренувальному датасеті(0,8228) значно відрізняється від точності на валідаційному(0,5783). Це означає, що модль не добре генералызує - модель перенавчена.


2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [4]:
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1, 25)}
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(X_train, train_targets)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [8]:
knn_best = knn_gs.best_estimator_
print("best params", knn_gs.best_params_)

best params {'n_neighbors': 24}


In [10]:
knn_train_pred = knn_best.predict_proba(X_train)[:, 1]
knn_val_pred = knn_best.predict_proba(X_val)[:, 1]

train_knn_auc = roc_auc_score(train_targets, knn_train_pred)
val_knn_auc = roc_auc_score(val_targets, knn_val_pred)

print(f"AUROC на тренувальних даних: {train_knn_auc:.4f}")
print(f"AUROC на валідаційних даних: {val_knn_auc:.4f}")

AUROC на тренувальних даних: 0.6885
AUROC на валідаційних даних: 0.6086


**Спостереження**

Порівняно з попередньою моделлю показники поліпшились: немає перенавчання, AUROC на валідційних даних також зріс.

Але порівняно з моделлю Дерева прийняття рішень результати гірші. При вирішенні проблеми за допомогою дерев вдалося досягнути AUROC 0,92 на валідаційному наборі.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [16]:
dt = DecisionTreeClassifier(random_state=42)
params_dt = {
    'max_depth': np.arange(1, 21,2),
    'max_leaf_nodes': np.arange(2,11,1)}

dt_gs = GridSearchCV(dt, params_dt, cv=3, scoring='roc_auc')
%time dt_gs.fit(X_train, train_targets)

CPU times: user 5.53 s, sys: 2.97 ms, total: 5.53 s
Wall time: 5.53 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                         'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])},
             scoring='roc_auc')

In [17]:
dt_best = dt_gs.best_estimator_
print("best params", dt_gs.best_params_)

best params {'max_depth': 5, 'max_leaf_nodes': 10}


In [19]:
dt_train_pred = dt_best.predict_proba(X_train)[:, 1]
dt_val_pred = dt_best.predict_proba(X_val)[:, 1]

train_dt_auc = roc_auc_score(train_targets, dt_train_pred)
val_dt_auc = roc_auc_score(val_targets, dt_val_pred)

print(f"AUROC на тренувальних даних: {train_dt_auc:.4f}")
print(f"AUROC на валідаційних даних: {val_dt_auc:.4f}")

AUROC на тренувальних даних: 0.9010
AUROC на валідаційних даних: 0.9018


**Спостереження**

Ця модель краща за попередню, але не краща за ту що ми знайшли вручну. Вручну ми досягли AUROC 0,92, а зараз лише 0,90

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [22]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

dt_random_search = RandomizedSearchCV(dt, params_dt, n_iter=40, cv=3, scoring='roc_auc', random_state=42)
%time dt_random_search.fit(X_train, train_targets)

CPU times: user 1.81 s, sys: 3.77 ms, total: 1.82 s
Wall time: 1.82 s


RandomizedSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=40,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'max_leaf_nodes': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'splitter': ['best', 'random']},
                   random_state=42, scoring='roc_auc')

In [23]:
dt_random_search_best = dt_random_search.best_estimator_
print("best params", dt_random_search.best_params_)

best params {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': 14, 'max_features': None, 'max_depth': 16, 'criterion': 'entropy'}


In [24]:
dt_random_search_train_pred = dt_random_search_best.predict_proba(X_train)[:, 1]
dt_random_search_val_pred = dt_random_search_best.predict_proba(X_val)[:, 1]

train_dt_random_search_auc = roc_auc_score(train_targets, dt_random_search_train_pred)
val_dt_random_search_auc = roc_auc_score(val_targets, dt_random_search_val_pred)

print(f"AUROC на тренувальних даних: {train_dt_random_search_auc:.4f}")
print(f"AUROC на валідаційних даних: {val_dt_random_search_auc:.4f}")

AUROC на тренувальних даних: 0.9166
AUROC на валідаційних даних: 0.9176


**Спостереження**

Порівняно з моделюю, яку ми занйшли за допомогою GridSearch показники моделі трішки поліпшились.

Тут ми додали більше параметрів в модель: criterion для вібіру функції розбиття дерева в вузлах, splitter визначає як алгоритм розбиває дані(best обирає тип розбиття, що дає найбільше зменшення невизначеності за критерієм Gini або Entropy; random випадково обирає можливі розбиття на підмножини, потім обирає найкреще з них), max_depth задає максимальну глибину дерева, max_leaf_nodes задає максимальну кількість листеів дерева, min_samples_split - мінімальна кількість зразків у вузлі для подальшого розбиття, min_samples_leaf - мінімальна кількість зразків у листку, max_features визначає скільки ознак буде розглядатись при кожному розбитті вузла.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [26]:
test_raw_df = pd.read_csv('test.csv')
test_inputs = preprocess_new_data(test_raw_df, data['scaler'], data['encoder'])
test_raw_df['Exited'] = dt_random_search_best.predict_proba(test_inputs)[:,1]

In [30]:
sample_submission_df = pd.read_csv('sample_submission.csv')
sample_submission_df['Exited'] = test_raw_df['Exited']
sample_submission_df.to_csv('submission_dt_random_search.csv', index=False)